In [8]:
# Load module
import pyomo.environ as pyo

# Define model
model = pyo.ConcreteModel()

# Define decision variables
product = ["PE", "PP"]
model.x = pyo.Var(product, domain=pyo.NonNegativeReals)

# Define objective function
profit = {"PE": 3, "PP": 2}
model.obj = pyo.Objective(expr=sum(profit[i]*model.x[i] for i in product), sense=pyo.maximize)

# Define constraints
raw_material = {
    "PE": {"C2H4": 1, "C3H6": 0},
    "PP": {"C2H4": 0, "C3H6": 1}
}
feed = ["C2H4", "C3H6"]
model.con_feed = pyo.Constraint(expr = sum(raw_material[p][f] * model.x[p] for p in product for f in feed) <= 80)

energy_demand = {
    "PE": 2,
    "PP": 1
}
model.con_energy = pyo.Constraint(expr = sum(energy_demand[p] * model.x[p] for p in product) <= 100)
model.con_PE_demand = pyo.Constraint(expr = model.x['PE'] <= 40)

solve = pyo.SolverFactory('ipopt')
solve.solve(model)

# print the results
print("Optimal solution:")
for i in product:
    print(f"{i}: {round(pyo.value(model.x[i]), 2)}")
print(f"Total profit: {round(pyo.value(model.obj), 2)}")


Optimal solution:
PE: 20.0
PP: 60.0
Total profit: 180.0
